<a href="https://colab.research.google.com/github/manula96/NLP-based-ChatBot-Covid19/blob/master/fanatic_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#install nltk package
pip install nltk

In [0]:
#install newspaper3k package 
pip install newspaper3k

In [0]:
#import libraries
from newspaper import Article
import random
import string 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer 
import nltk
import numpy as np
import warnings

In [0]:
#Ignore any warnings
warnings.filterwarnings('ignore')

In [0]:
#Download the packages from NLTK
nltk.download('punkt',quiet=True)
nltk.download('wordnet',quiet=True)

True

In [0]:
#Get the article URL
#article = Article('https://www.cnet.com/how-to/coronavirus-explained-symptoms-lockdowns-and-all-your-covid-19-questions-answered/')
article = Article('https://www.mayoclinic.org/diseases-conditions/coronavirus/symptoms-causes/syc-20479963')
article.download()
article.parse()
article.nlp()
corpus = article.text

#Print to validate
print(corpus)


Overview

Coronavirus Open pop-up dialog box Close Coronavirus Coronavirus Coronavirus is a family of viruses that can cause respiratory illnesses such as the common cold, severe acute respiratory syndrome (SARS) and Middle East respiratory syndrome (MERS).

Coronaviruses are a family of viruses that can cause illnesses such as the common cold, severe acute respiratory syndrome (SARS) and Middle East respiratory syndrome (MERS). In 2019, a new coronavirus was identified as the cause of a disease outbreak that originated in China.

The virus is now known as the severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The disease it causes is called coronavirus disease 2019 (COVID-19). In March 2020, the World Health Organization (WHO) declared the COVID-19 outbreak a pandemic.

Public health groups, including the U.S. Centers for Disease Control and Prevention (CDC) and WHO , are monitoring the pandemic and posting updates on their websites. These groups have also issued recommenda

In [0]:
#tokenization
text = corpus
sent_tokens = nltk.sent_tokenize(text) #to sentences

#print sentences
print(sent_tokens)

['Overview\n\nCoronavirus Open pop-up dialog box Close Coronavirus Coronavirus Coronavirus is a family of viruses that can cause respiratory illnesses such as the common cold, severe acute respiratory syndrome (SARS) and Middle East respiratory syndrome (MERS).', 'Coronaviruses are a family of viruses that can cause illnesses such as the common cold, severe acute respiratory syndrome (SARS) and Middle East respiratory syndrome (MERS).', 'In 2019, a new coronavirus was identified as the cause of a disease outbreak that originated in China.', 'The virus is now known as the severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).', 'The disease it causes is called coronavirus disease 2019 (COVID-19).', 'In March 2020, the World Health Organization (WHO) declared the COVID-19 outbreak a pandemic.', 'Public health groups, including the U.S. Centers for Disease Control and Prevention (CDC) and WHO , are monitoring the pandemic and posting updates on their websites.', 'These groups have 

In [0]:
#create dictionary to remove punctuations
remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation)

#print
print(string.punctuation)

print(remove_punct_dict)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [0]:
#create a function to return a list of limitized lowercase words after removing punctuations
def LemNormalize(text):
  return nltk.word_tokenize(text.lower().translate(remove_punct_dict))

#print the tokenized text
print(LemNormalize(text))

['overview', 'coronavirus', 'open', 'popup', 'dialog', 'box', 'close', 'coronavirus', 'coronavirus', 'coronavirus', 'is', 'a', 'family', 'of', 'viruses', 'that', 'can', 'cause', 'respiratory', 'illnesses', 'such', 'as', 'the', 'common', 'cold', 'severe', 'acute', 'respiratory', 'syndrome', 'sars', 'and', 'middle', 'east', 'respiratory', 'syndrome', 'mers', 'coronaviruses', 'are', 'a', 'family', 'of', 'viruses', 'that', 'can', 'cause', 'illnesses', 'such', 'as', 'the', 'common', 'cold', 'severe', 'acute', 'respiratory', 'syndrome', 'sars', 'and', 'middle', 'east', 'respiratory', 'syndrome', 'mers', 'in', '2019', 'a', 'new', 'coronavirus', 'was', 'identified', 'as', 'the', 'cause', 'of', 'a', 'disease', 'outbreak', 'that', 'originated', 'in', 'china', 'the', 'virus', 'is', 'now', 'known', 'as', 'the', 'severe', 'acute', 'respiratory', 'syndrome', 'coronavirus', '2', 'sarscov2', 'the', 'disease', 'it', 'causes', 'is', 'called', 'coronavirus', 'disease', '2019', 'covid19', 'in', 'march', '

In [0]:
#keyword maching
#greeting inputs for users
GREETINGS_INPUTS = ["hi", "hello","greetings","hey"]

#greeting responses
GREETNGS_RESPONSE = ["hi", "hey there","hello"]

#return a random G response
def greeting(sentence):
  for word in sentence.split():
    if word.lower()in GREETINGS_INPUTS:
      return random.choice(GREETNGS_RESPONSE)





In [0]:
#generate the response
def response(user_query):
  #users query
  #user_query = 'What are the symptoms?'
  user_query = user_query.lower()

  #print(user_query)

  #set the chatbot response
  bot_response = ''

  #append the user query to the sentence list
  sent_tokens.append(user_query)
  #print(sent_tokens) ##print after appending the user query to the bot response

  #create a TfidfVectorizer object

  TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')

  #convert text to a metrix of TF-IDF (frequency an inverse document) features
  #TF is frequency of a term occurence in a document IDF-how rare thus TFIDF=TF x IDF

  tfidf = TfidfVec.fit_transform(sent_tokens)

  #print(tfidf) ##prnt the tfidf metrix

  #most similar score index from the article for the user query

  vals = cosine_similarity(tfidf[-1], tfidf)
  #print(vals) #print similarity score

  idx = vals.argsort()[0][-2]
  #print(idx)

  #reduce the dimensionality of vals

  flat = vals.flatten()

  #sort the list in ascending order
  flat.sort()

  #get the most similar score to the uers respone
  score = flat[-2]
  #print(score)

  #if the cariable score is 0 then there is no text similar to the users query.
  if(score == 0):
    bot_response+"I apologise, I do not understand"
  else:
    bot_response = bot_response+sent_tokens[idx]

  #print bot response
  #print(bot_response)

  #remove appended question from response
  sent_tokens.remove(user_query)
  
  return bot_response



In [0]:
flag = True
print("BotRona: I will answer your queries about Covid19 (Corona). If you want to exit, type 'bye'")

while(flag == True):
  user_query = input()
  user_query = user_query.lower()
  if (user_query != 'bye'):
    if(user_query == 'thanks' or user_query == 'thank you'):
      flag=False
      print("BotRona: You are welcome!") 
    else:
      if(greeting(user_query) != None):
        print("BotRona: "+greeting(user_query))
      else:
        print("BotRona: "+response(user_query))
        
  else:
    flag = False
    print("BotRona: Hope You got the answers! Stay safe.")



BotRona: I will answer your queries about Covid19 (Corona). If you want to exit, type 'bye'
hi
BotRona: hey there
whats covid19
BotRona: Keep in mind some people may have COVID-19 and spread it to others, even if they don't have symptoms or don't know they have COVID-19 .
symptoms?
BotRona: Some people may have only a few symptoms, and some people may have no symptoms at all.
what are the symptoms
BotRona: Some people may have only a few symptoms, and some people may have no symptoms at all.
common signs
BotRona: If you have emergency COVID-19 signs and symptoms, seek care immediately.
other symptoms
BotRona: Some people may have only a few symptoms, and some people may have no symptoms at all.
when to see a doctor?
BotRona: When to see a doctor

If you have COVID-19 symptoms or you've been in contact with someone diagnosed with COVID-19 , contact your doctor or clinic right away for medical advice.
causes?
BotRona: Causes

Infection with the new coronavirus (severe acute respiratory s